This post contains some notes about three Python libraries for working with numerical data too large to fit into main memory: [``h5py``](http://www.h5py.org/), [``bcolz``](http://bcolz.blosc.org/) and [``zarr``](https://github.com/alimanfoo/zarr).

## HDF5 (``h5py``)

When I first discovered the [HDF5 file format](https://www.hdfgroup.org/HDF5/) a few years ago it was pretty transformative. I'd been struggling to find efficient ways of exploring and analysing data coming from [our research](http://www.malariagen.net/ag1000g) on genetic variation in the mosquitoes that carry malaria. The data are not enormous - typically integer arrays with around 20 billion elements - but they are too large to work with in memory on a typical laptop or desktop machine. The file formats traditionally used for these data are very slow to parse, so I went looking for an alternative.

HDF5 files provide a great solution for storing multi-dimensional arrays of numerical data. The arrays are divided up into chunks and each chunk is compressed, enabling data to be stored efficiently in memory or on disk. Depending on how the chunks are configured, usually a good compromise can be achieved that means data can be read very quickly even when using different access patterns, e.g., taking horizontal or vertical slices of a matrix. Also, the [``h5py``](http://www.h5py.org/) Python library provides a very convenient API for working with HDF5 files. I found there was a whole range of analyses I could happily get done on my laptop on the train home from work.

## ``bcolz.carray``

A bit later on I discovered the [``bcolz``](http://bcolz.blosc.org/) library. ``bcolz`` is primarily intended for storing and querying large tables of data, but it does provide a [``carray``](http://bcolz.blosc.org/reference.html#the-carray-class) class that is roughly analogous to an HDF5 dataset in that it can store numerical arrays in a chunked, compressed form, either in memory or on disk. 

Reading and writing data to a ``carray`` is typically a lot faster than HDF5. For example:

In [1]:
import numpy as np
import h5py
import bcolz
import tempfile


def h5fmem(**kwargs):
    """Convenience function to create an in-memory HDF5 file."""

    # need a file name even tho nothing is ever written
    fn = tempfile.mktemp()

    # file creation args
    kwargs['mode'] = 'w'
    kwargs['driver'] = 'core'
    kwargs['backing_store'] = False

    # open HDF5 file
    h5f = h5py.File(fn, **kwargs)

    return h5f

Setup a simple array of integer data to store.

In [2]:
a1 = np.arange(1e8, dtype='i4')
a1

array([       0,        1,        2, ..., 99999997, 99999998, 99999999], dtype=int32)

Time how long it takes to store in an HDF5 dataset.

In [3]:
%timeit h5fmem().create_dataset('arange', data=a1, chunks=(2**18,), compression='gzip', compression_opts=1, shuffle=True)

1 loop, best of 3: 1.4 s per loop


Time how long it takes to store in a ``carray``.

In [4]:
%timeit bcolz.carray(a1, chunklen=2**18, cparams=bcolz.cparams(cname='lz4', clevel=5, shuffle=1))

10 loops, best of 3: 108 ms per loop


In the example above, ``bcolz`` is more than 10 times faster at storing (compressing) the data than HDF5. As I understand it, this performance gain comes from several factors. ``bcolz`` uses a C library called [``blosc``](https://github.com/blosc/c-blosc) internally to perform compression and decompression operations. ``blosc`` can use multiple threads, so some of the work is done in parallel. ``blosc`` also splits data up in a way that is designed to work well with the CPU cache architecture. Finally, ``blosc`` is a meta-compressor and several different compression libraries can be used - above I used the ``lz4`` compressor, which does not achieve quite the same compression ratios as ``gzip`` (``zlib``) but is much faster with numerical data.

## ``zarr``

Speed really makes a difference when working interactively with data, so I started using the ``carray`` class where possible in my analyses, especially for storing intermediate data. However, it does have some limitations. A ``carray`` can be multidimensional, but because ``bcolz`` is not really designed for multi-dimensional data, a ``carray`` can only be chunked along the first dimension. This means taking slices of the first dimension is efficient, but slicing any other dimension will be very inefficient, because the entire array will need to be read and decompressed to access even a single column of a matrix.

To explore better ways of working with large multi-dimensional data, I recently created a new library called [``zarr``](https://github.com/alimanfoo/zarr). ``zarr`` borrows code heavily from ``bcolz``, and in particular it also uses ``blosc`` internally to handle all compression and decompression operations. However, ``zarr`` supports chunking of arrays along multiple dimensions, enabling good performance for multiple data access patterns. For example:

In [5]:
import zarr

Setup a 2-dimensional array of integer data.

In [6]:
a2 = np.arange(1e8, dtype='i4').reshape(10000, 10000)

Store the data in a ``carray``.

In [7]:
c2 = bcolz.carray(a2, chunklen=100)
c2

carray((10000, 10000), int32)
  nbytes: 381.47 MB; cbytes: 10.63 MB; ratio: 35.87
  cparams := cparams(clevel=5, shuffle=1, cname='blosclz')
[[       0        1        2 ...,     9997     9998     9999]
 [   10000    10001    10002 ...,    19997    19998    19999]
 [   20000    20001    20002 ...,    29997    29998    29999]
 ..., 
 [99970000 99970001 99970002 ..., 99979997 99979998 99979999]
 [99980000 99980001 99980002 ..., 99989997 99989998 99989999]
 [99990000 99990001 99990002 ..., 99999997 99999998 99999999]]

Store the data in a ``zarr`` array.

In [8]:
z = zarr.array(a2, chunks=(1000, 1000))
z

zarr.ext.SynchronizedArray((10000, 10000), int32, chunks=(1000, 1000))
  cname: blosclz; clevel: 5; shuffle: 1 (BYTESHUFFLE)
  nbytes: 381.5M; cbytes: 10.0M; ratio: 38.0; initialized: 100/100

Time how long it takes to access a slice along the first dimension.

In [9]:
%timeit c2[:1000]

100 loops, best of 3: 12.4 ms per loop


In [10]:
%timeit z[:1000]

10 loops, best of 3: 19.2 ms per loop


Time how long it takes to access a slice along the second dimension.

In [11]:
%timeit c2[:, :1000]

10 loops, best of 3: 130 ms per loop


In [12]:
%timeit z[:, :1000]

100 loops, best of 3: 12.7 ms per loop


By using ``zarr`` and chunking along both dimensions of the array, we have forfeited a small amount of speed when slicing the first dimension to gain a lot of speed when accessing the second dimension.

Like ``h5py`` and ``bcolz``, ``zarr`` can store data either in memory or on disk. However, ``zarr`` has some unique features too. For example, multi-dimensional arrays can be resized along any dimension, allowing an array to be grown by appending new data in a flexible way. Also, ``zarr`` arrays can be used in parallel computations, supporting concurrent reads and writes in either a multi-threaded or multi-process context. That is something I am just beginning to explore, and hope to follow up in a separate post.

``zarr`` is still in an experimental phase, but if you do try it out, any feedback is very welcome.

## Further reading

* [HDF5](https://www.hdfgroup.org/HDF5/)
* [h5py](http://www.h5py.org/)
* [bcolz](http://bcolz.blosc.org/)
* [blosc](http://blosc.org/)
* [zarr](https://github.com/alimanfoo/zarr)

## Post-script: Performance with real genotype data

Here are some benchmarks with real genotype data.

In [13]:
import operator
from functools import reduce


def human_readable_size(size):
    if size < 2**10:
        return "%s" % size
    elif size < 2**20:
        return "%.1fK" % (size / float(2**10))
    elif size < 2**30:
        return "%.1fM" % (size / float(2**20))
    elif size < 2**40:
        return "%.1fG" % (size / float(2**30))
    else:
        return "%.1fT" % (size / float(2**40))

    
def h5d_diagnostics(d):
    """Print some diagnostics on an HDF5 dataset."""
    
    print(d)
    nbytes = reduce(operator.mul, d.shape) * d.dtype.itemsize
    cbytes = d._id.get_storage_size()
    if cbytes > 0:
        ratio = nbytes / cbytes
    else:
        ratio = np.inf
    r = '  cname=%s' % d.compression
    r += ', clevel=%s' % d.compression_opts
    r += ', shuffle=%s' % d.shuffle
    r += '\n  nbytes=%s' % human_readable_size(nbytes)
    r += ', cbytes=%s' % human_readable_size(cbytes)
    r += ', ratio=%.1f' % ratio
    r += ', chunks=%s' % str(d.chunks)
    print(r)
    

Locate a genotype dataset within an HDF5 file from the [Ag1000G project](http://www.malariagen.net/data/ag1000g-phase1-AR3).

In [14]:
callset = h5py.File('/data/coluzzi/ag1000g/data/phase1/release/AR3/variation/main/hdf5/ag1000g.phase1.ar3.pass.h5',
                    mode='r')
genotype = callset['3R/calldata/genotype']
genotype

<HDF5 dataset "genotype": shape (13167162, 765, 2), type "|i1">

Extract the first million rows of the dataset to use for benchmarking.

In [15]:
a3 = genotype[:1000000]

Benchmark compression performance.

In [16]:
%%time
genotype_h5dmem = h5fmem().create_dataset('genotype', data=a3, 
                                          compression='gzip', compression_opts=1, shuffle=False,
                                          chunks=(10000, 100, 2))

CPU times: user 6.76 s, sys: 24 ms, total: 6.79 s
Wall time: 6.76 s


In [17]:
h5d_diagnostics(genotype_h5dmem)

<HDF5 dataset "genotype": shape (1000000, 765, 2), type "|i1">
  cname=gzip, clevel=1, shuffle=False
  nbytes=1.4G, cbytes=51.1M, ratio=28.5, chunks=(10000, 100, 2)


In [18]:
%%time
genotype_carray = bcolz.carray(a3, cparams=bcolz.cparams(cname='lz4', clevel=1, shuffle=2))

CPU times: user 2.4 s, sys: 28 ms, total: 2.43 s
Wall time: 778 ms


In [19]:
genotype_carray

carray((1000000, 765, 2), int8)
  nbytes: 1.42 GB; cbytes: 48.70 MB; ratio: 29.96
  cparams := cparams(clevel=1, shuffle=2, cname='lz4')
[[[0 0]
  [0 0]
  [0 0]
  ..., 
  [0 0]
  [0 0]
  [0 0]]

 [[0 0]
  [0 0]
  [0 0]
  ..., 
  [0 0]
  [0 0]
  [0 0]]

 [[0 0]
  [0 0]
  [0 0]
  ..., 
  [0 0]
  [0 0]
  [0 0]]

 ..., 
 [[0 0]
  [0 0]
  [0 0]
  ..., 
  [0 0]
  [0 0]
  [0 0]]

 [[0 0]
  [0 0]
  [0 0]
  ..., 
  [0 0]
  [0 0]
  [0 0]]

 [[0 0]
  [0 0]
  [0 0]
  ..., 
  [0 0]
  [0 0]
  [0 0]]]

In [20]:
%%time
genotype_zarr = zarr.array(a3, chunks=(10000, 100, 2), cname='lz4', clevel=1, shuffle=2)

CPU times: user 2.7 s, sys: 68 ms, total: 2.77 s
Wall time: 1.19 s


In [21]:
genotype_zarr

zarr.ext.SynchronizedArray((1000000, 765, 2), int8, chunks=(10000, 100, 2))
  cname: lz4; clevel: 1; shuffle: 2 (BITSHUFFLE)
  nbytes: 1.4G; cbytes: 50.2M; ratio: 29.1; initialized: 800/800

Note that although I've used the LZ4 compression library with ``bcolz`` and ``zarr``, the compression ratio is actually better than when using gzip (zlib) with HDF5. This is due to the bitshuffle filter, which comes bundled with ``bcolz`` and ``zarr``. The bitshuffle filter can also be used with HDF5 with some configuration I believe.

Similar to the synthetic ``arange`` dataset, compression with ``bcolz`` is around 10 times faster than with HDF5, due to a combination of factors. Compression with ``zarr`` is slightly slower than ``bcolz``, but this is entirely due to the choice of chunk shape and the correlation structure in the data. If we use the same chunking for both, compression speed is similar...

In [22]:
%%time 
_ = zarr.array(a3, chunks=(genotype_carray.chunklen, 765, 2), cname='lz4', clevel=1, shuffle=2)

CPU times: user 2.1 s, sys: 36 ms, total: 2.14 s
Wall time: 667 ms


Benchmark data access via slices along first and second dimensions. 

In [23]:
%timeit genotype_h5dmem[:10000]

10 loops, best of 3: 24.5 ms per loop


In [24]:
%timeit genotype_carray[:10000]

100 loops, best of 3: 10.8 ms per loop


In [25]:
%timeit genotype_zarr[:10000]

100 loops, best of 3: 14.3 ms per loop


In [26]:
%timeit genotype_h5dmem[:, :10]

1 loop, best of 3: 284 ms per loop


In [27]:
%timeit genotype_carray[:, :10]

1 loop, best of 3: 1.63 s per loop


In [28]:
%timeit genotype_zarr[:, :10]

10 loops, best of 3: 128 ms per loop


## Setup

In [29]:
# run on my laptop, which doesn't have AVX2
import cpuinfo
cpuinfo.main()

Vendor ID: GenuineIntel
Hardware Raw: 
Brand: Intel(R) Core(TM) i7-3667U CPU @ 2.00GHz
Hz Advertised: 2.0000 GHz
Hz Actual: 3.0374 GHz
Hz Advertised Raw: (2000000000, 0)
Hz Actual Raw: (3037402000, 0)
Arch: X86_64
Bits: 64
Count: 4
Raw Arch String: x86_64
L2 Cache Size: 4096 KB
L2 Cache Line Size: 0
L2 Cache Associativity: 0
Stepping: 9
Model: 58
Family: 6
Processor Type: 0
Extended Model: 0
Extended Family: 0
Flags: acpi, aes, aperfmperf, apic, arat, arch_perfmon, avx, bts, clflush, cmov, constant_tsc, cx16, cx8, de, ds_cpl, dtes64, dtherm, dts, eagerfpu, epb, ept, erms, est, f16c, flexpriority, fpu, fsgsbase, fxsr, ht, ida, lahf_lm, lm, mca, mce, mmx, monitor, msr, mtrr, nonstop_tsc, nopl, nx, pae, pat, pbe, pcid, pclmulqdq, pdcm, pebs, pge, pln, pni, popcnt, pse, pse36, pts, rdrand, rdtscp, rep_good, sep, smep, smx, ss, sse, sse2, sse4_1, sse4_2, ssse3, syscall, tm, tm2, tpr_shadow, tsc, tsc_deadline_timer, vme, vmx, vnmi, vpid, x2apic, xsave, xsaveopt, xtopology, xtpr
